# Homework 3

In [63]:
# import libraries:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
# Other required packages
# Modules for data-analysis
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats import multicomp
import xlrd

##### The daily energy intake from 11 healthy women is [5260., 5470., 5640., 6180., 6390., 6515., 6805., 7515., 7515., 8230., 8770.] kJ. Is this value significantly different from the recommended value of 7725?

In [9]:
data = [5260., 5470., 5640., 6180., 6390., 6515., 6805., 7515., 7515., 8230., 8770.]
check_value = 7725

t, pVal = stats.ttest_1samp(data, check_value)
print("p_test =", pVal)


p_test = 0.018137235176105812


Yes, this value is significantly different as the p value is equal to 0.018.

##### In a clinic, 15 lazy patients weigh [76, 101, 66, 72, 88, 82, 79, 73, 76, 85, 75, 64, 76, 81, 86] kg, and 15 sporty patients weigh [ 64, 65, 56, 62, 59, 76, 66, 82, 91, 57, 92, 80, 82, 67, 54] kg.

**Are the lazy patients significantly heavier?**

In [27]:
# load data
lazy = [76, 101, 66, 72, 88, 82, 79, 73, 76, 85, 75, 64, 76, 81, 86]
sporty = [64, 65, 56, 62, 59, 76, 66, 82, 91, 57, 92, 80, 82, 67, 54]

# run t-test
t_statistic, pVal = stats.ttest_ind(lazy, sporty)
print("p_test =", pVal)

p_test = 0.04516066585880743


Yes, the lazy patients are significantly heavier from the p value of 0.045.

**Are the two data sets normally distributed?**

Yes, the two data sets are normally distributed.

In [61]:
for ii, data in enumerate((lazy, sporty)):
    (_, pval) = stats.normaltest(data)
if pval > 0.05:
    print('Dataset # {0} is normally distributed'.format(ii))

Dataset # 1 is normally distributed


**Are the lazy patients still heavier, if you check with the Mann–Whitney test?**

In [40]:
u, p_value = stats.mannwhitneyu(lazy, sporty, alternative='two-sided')
print("p_test =", p_value)

p_test = 0.07740194475805182


Since p > 0.05, the lazy patients aren't heavier with the Mann-Whitney test.

##### Perform an ANOVA. Are the three groups different?

In [46]:
df = pd.read_excel("Table 6.6 Plant experiment.xls", skiprows=2, usecols=[0, 1, 2]).dropna()
df

,control,treatmentA,treatmentB
0,4.17,4.81,6.31
1,5.58,4.17,5.12
2,5.18,4.41,5.54
3,6.11,3.59,5.50
4,4.50,5.87,5.37
5,4.61,3.83,5.29
6,5.17,6.03,4.92
7,4.53,4.89,6.15
8,5.33,4.32,5.80
9,5.14,4.69,5.26


In [48]:
F_statistic, pVal = stats.f_oneway(df['control'], df['treatmentA'], df['treatmentB'])
print("p_test =", pVal)

p_test = 0.015909958325622895


As p value < 0.05, the three groups are different.

#### Using the Tukey test, which of the pairs are different?

In [72]:
# function to do tukey tests
def compare_many(data):
    '''Multiple comparisons: Which one is different? '''
    print('\n MultComp:--------------------------------------')
    mc = multicomp.MultiComparison(data['weight'], data['group'])
    print(mc.tukeyhsd().summary())
    # Show the group names
    print(mc.groupsunique)
    # Generate a print ----------------
    res2 = mc.tukeyhsd() # Get the data
    xvals = np.arange(3)

def get_ANOVA_data():
    inFile = 'Table 6.6 Plant experiment.xls' 
    book = xlrd.open_workbook(inFile)
    sheet = book.sheet_by_index(0)
    treatment = sheet.col_values(4)
    weight = sheet.col_values(5)
    data = pd.DataFrame({'group':treatment[3:], 'weight':
    weight[3:]})
    return data

In [73]:
# do pairwise tests
data = get_ANOVA_data()
compare_many(data)


 MultComp:--------------------------------------
    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1     group2   meandiff p-adj   lower  upper  reject
-----------------------------------------------------------
   Control TreatmentA   -0.371 0.3909 -1.0622 0.3202  False
   Control TreatmentB    0.494  0.198 -0.1972 1.1852  False
TreatmentA TreatmentB    0.865  0.012  0.1738 1.5562   True
-----------------------------------------------------------
['Control' 'TreatmentA' 'TreatmentB']


From the Tukey test above, we can see that Treatment A and Treatment B are different from the Control.

#### Would a nonparametric comparison lead to a different result?

In [75]:
def KruskalWallis(data):
    g_a = data['weight'][data['group']=='TreatmentA']
    g_b = data['weight'][data['group']=='TreatmentB']
    g_c = data['weight'][data['group']=='Control']

    h, p = stats.kruskal(g_c, g_a, g_b)
    print('Result from Kruskal-Wallis test: p = {0}'.format(p))

In [76]:
# do kruskal wallis tests
KruskalWallis(data)

Result from Kruskal-Wallis test: p = 0.018423755731471963


Since p < 0.05, even this test says that the data is different from each other. 